In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 8)
import warnings; warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from sympy import symbols

import sys; sys.path.append('../')
from PDE_FIND import *

import sys; sys.path.append('../../parametric-discovery/')
from best_subset import *
from frols import frols
from p_linear_regression import PLinearRegression
from r_pca import R_pca
from pde_diff_new import RPCA
from RobustPCA.rpca import RobustPCA

import scipy.io as sio
from scipy.signal import savgol_filter
import itertools

from sklearn.preprocessing import normalize
from sklearn.linear_model import Ridge, BayesianRidge, LinearRegression as SkLinearRegression
from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from abess.linear import LinearRegression
import pysindy as ps
from solvel0 import solvel0, miqp, miqp2, miqp3

from tqdm import trange
from os.path import join as join_path
FIGURE_EXPORT_PATH = "/Users/pongpisit/Documents/figure_export/"

np.random.seed(100)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Sklearn's version: 1.2.2
mrmr is not installed in the env you are using. This may cause an error in future if you try to use the (missing) lib.


In [2]:
def check_percent(bic_scores, complexities):
    slope = (bic_scores[1:]-bic_scores[:-1])/(complexities[1:]-complexities[:-1])
    slope_index = np.argmin(slope)
    percent_improve = 100*np.abs(bic_scores[slope_index+1]-bic_scores[slope_index])/np.abs(bic_scores[slope_index])
    percent_from_1 = 100*np.abs(bic_scores[slope_index+1]-bic_scores[0])/np.abs(bic_scores[0])
    return slope_index+1, percent_improve, percent_from_1

def percent_coeff(pred, ground):
    return 100*(np.abs(pred-ground)/np.abs(ground))

In [3]:
n = 128
data = sio.loadmat(f"../Datasets/reaction_diffusion_3d_{n}.mat")
u_sol = real(data['usol'])
v_sol = real(data['vsol'])
x = real(data['x'][0])
y = real(data['y'][0])
z = real(data['z'][0])
t = real(data['t'][0])

del data

dt = t[1] - t[0]
dx = x[1] - x[0]
dy = y[1] - y[0]
dz = z[1] - z[0]
# For WeakPDELibrary
ground_indices_u = (0, 1, 5, 9, 15, 23)
ground_coeff_u = np.array([0.014, -0.014, -1.000, 0.020, 0.020, 0.020])
ground_indices_v = (2, 5, 10, 16, 24)
ground_coeff_v = np.array([-0.067, 1.0, 0.01, 0.01, 0.01])

x, y, z, t

(array([-1.25      , -1.23046875, -1.2109375 , -1.19140625, -1.171875  ,
        -1.15234375, -1.1328125 , -1.11328125, -1.09375   , -1.07421875,
        -1.0546875 , -1.03515625, -1.015625  , -0.99609375, -0.9765625 ,
        -0.95703125, -0.9375    , -0.91796875, -0.8984375 , -0.87890625,
        -0.859375  , -0.83984375, -0.8203125 , -0.80078125, -0.78125   ,
        -0.76171875, -0.7421875 , -0.72265625, -0.703125  , -0.68359375,
        -0.6640625 , -0.64453125, -0.625     , -0.60546875, -0.5859375 ,
        -0.56640625, -0.546875  , -0.52734375, -0.5078125 , -0.48828125,
        -0.46875   , -0.44921875, -0.4296875 , -0.41015625, -0.390625  ,
        -0.37109375, -0.3515625 , -0.33203125, -0.3125    , -0.29296875,
        -0.2734375 , -0.25390625, -0.234375  , -0.21484375, -0.1953125 ,
        -0.17578125, -0.15625   , -0.13671875, -0.1171875 , -0.09765625,
        -0.078125  , -0.05859375, -0.0390625 , -0.01953125,  0.        ,
         0.01953125,  0.0390625 ,  0.05859375,  0.0

In [4]:
time = t
u = np.zeros((n, n, n, len(time), 2))
u[:, :, :, :, 0] = u_sol
u[:, :, :, :, 1] = v_sol
# u_dot = ps.FiniteDifference(axis=3)._differentiate(u, dt)

# Odd polynomial terms in (u, v), up to second order derivatives in (u, v)
library_functions = [
    lambda x: x,
    lambda x: x * x * x,
    lambda x, y: x * y * y,
    lambda x, y: x * x * y,
]
library_function_names = [
    lambda x: x,
    lambda x: x + x + x,
    lambda x, y: x + y + y,
    lambda x, y: x + x + y,
]

# Need to define the 2D spatial grid before calling the library
X, Y, Z, T = np.meshgrid(x, y, z, time, indexing="ij")
spatiotemporal_grid = np.asarray([X, Y, Z, T])
spatiotemporal_grid = np.transpose(spatiotemporal_grid, axes=[1, 2, 3, 4, 0])
weak_lib = ps.WeakPDELibrary(
    library_functions=library_functions,
    function_names=library_function_names,
    derivative_order=2,
    spatiotemporal_grid=spatiotemporal_grid,
    is_uniform=True,
    include_interaction=False,
    include_bias=True,
    periodic=True,
    K=10000,
    cache=True
)

In [5]:
np.random.seed(100)
noise_lvs = [0, 0.04, 0.1]
noise_lv = noise_lvs[-1]
domain_noise = 0.01*np.abs(noise_lv)*np.std(u)*np.random.randn(*u.shape)
u = u + domain_noise
# np.save("u_noise1e-1.npy", u)

In [6]:
denoise = True
if denoise: 
    u = np.load("u_noise1e-1_denoised.npy")

In [7]:
target_index = 1
max_complexity = 12
X_pre = np.load("./IPI_output_files/X_pre_noise1e-1_denoised.npy")
y_pre = np.load("./IPI_output_files/y_pre_noise1e-1_denoised.npy")

In [9]:
solvel0(X_pre, y_pre[:, target_index:target_index+1], 
        is_normal=True, miosr=False, refine=True, 
        max_complexity=max_complexity)

100%|█████████████████████████████████████████████████████████████████████████████████████| 12/12 [04:06<00:00, 20.56s/it]


Call backward_refinement...


[(2,),
 (16, 24),
 (5, 16, 24),
 (2, 5, 16, 24),
 (2, 5, 16, 23, 24),
 (2, 5, 6, 16, 23, 24),
 (2, 5, 6, 15, 16, 23, 24),
 (2, 4, 5, 6, 15, 16, 23, 24),
 (1, 2, 4, 5, 6, 15, 16, 23, 24),
 (1, 2, 3, 4, 5, 6, 15, 16, 23, 24),
 (0, 1, 2, 3, 4, 5, 6, 15, 16, 23, 24),
 (0, 1, 2, 3, 4, 5, 12, 15, 16, 17, 21, 24)]

In [10]:
solvel0(X_pre, y_pre[:, target_index:target_index+1], 
        is_normal=True, miosr=True, refine=True, 
        max_complexity=max_complexity)

100%|█████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 40.54it/s]


Call backward_refinement...


[(2,), (16, 24), (5, 16, 24), (2, 5, 16, 24), (2, 5, 10, 16, 24)]

In [33]:
miosr_selected_inds_list = []
for ts in range(1, max_complexity+1):
    optimizer = ps.MIOSR(target_sparsity=ts, alpha=5e-2, normalize_columns=True, 
                         regression_timeout=100)
    optimizer.fit(X_pre, y_pre[:, target_index:target_index+1])
    miosr_selected_inds_list.append(tuple(np.nonzero(optimizer.coef_[0])[0]))
miosr_selected_inds_list, reduce(np.union1d, miosr_selected_inds_list)

([(2,),
  (16, 24),
  (5, 16, 24),
  (2, 5, 16, 24),
  (2, 5, 10, 16, 24),
  (1, 2, 5, 10, 16, 24),
  (1, 2, 3, 5, 10, 16, 24),
  (1, 2, 3, 5, 10, 15, 16, 24),
  (1, 2, 3, 5, 10, 15, 16, 22, 24),
  (1, 2, 3, 5, 9, 10, 15, 16, 22, 24),
  (1, 2, 3, 4, 5, 10, 15, 16, 21, 22, 24),
  (0, 1, 2, 3, 4, 5, 10, 15, 16, 21, 22, 24)],
 array([ 0,  1,  2,  3,  4,  5,  9, 10, 15, 16, 21, 22, 24]))

In [34]:
frols_coeffs,selected_inds_list,_,_,_ = frols(y_pre[:, target_index:target_index+1], X_pre, 
                                              max_nonzeros=max_complexity)
frols_selected_inds_list = sorted(selected_inds_list, key=len)
frols_selected_inds_list, reduce(np.union1d, frols_selected_inds_list)

([(2,),
  (2, 24),
  (2, 5, 24),
  (2, 5, 16, 24),
  (2, 5, 10, 16, 24),
  (2, 5, 10, 16, 23, 24),
  (2, 5, 10, 11, 16, 23, 24),
  (2, 5, 10, 11, 16, 18, 23, 24),
  (2, 5, 8, 10, 11, 16, 18, 23, 24),
  (2, 5, 8, 9, 10, 11, 16, 18, 23, 24),
  (2, 5, 8, 9, 10, 11, 16, 17, 18, 23, 24),
  (2, 3, 5, 8, 9, 10, 11, 16, 17, 18, 23, 24)],
 array([ 2,  3,  5,  8,  9, 10, 11, 16, 17, 18, 23, 24]))

In [41]:
ever = reduce(np.union1d, miosr_selected_inds_list)
ever

array([ 0,  1,  2,  3,  4,  5,  9, 10, 15, 16, 21, 22, 24])

In [42]:
_, bs = brute_force_all_subsets(X_pre[:, ever], 
                                y_pre[:, target_index:target_index+1], 
                                max_support_size=12)
best_subsets = [tuple(ever[list(b)]) for b in bs]
del bs
best_subsets

100%|█████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:06<00:00,  1.95it/s]


[(2,),
 (16, 24),
 (5, 16, 24),
 (2, 5, 16, 24),
 (2, 5, 10, 16, 24),
 (2, 4, 5, 10, 16, 24),
 (2, 4, 5, 9, 10, 16, 24),
 (2, 4, 5, 9, 10, 16, 21, 24),
 (1, 2, 4, 5, 9, 10, 16, 21, 24),
 (1, 2, 3, 4, 5, 9, 10, 16, 21, 24),
 (1, 2, 3, 4, 5, 9, 10, 16, 21, 22, 24),
 (0, 1, 2, 3, 4, 5, 9, 10, 16, 21, 22, 24)]

In [43]:
from UBIC import UBIC, UBICs, baye_uncertainties, decreasing_values_indices, log_like_value

In [44]:
tau = 3
thres = 0.02

post_means, b_bics, b_uns = baye_uncertainties(best_subsets, (X_pre, y_pre[:, target_index:target_index+1]), u_type='cv1', take_sqrt=True)
predictions = X_pre@post_means
lower_bounds = []
for k, efi in enumerate(best_subsets):
    lower_bound = log_like_value(predictions[:, k:k+1], y_pre[:, 0:1])/np.log(len(y_pre[:, target_index:target_index+1]))-len(efi)
    lower_bounds.append(lower_bound)
    
last_lam = np.log10(max(lower_bounds/b_uns))
delta = last_lam/tau
now_lam = last_lam-delta
last_ubic = UBIC(b_bics, b_uns, len(y_pre[:, target_index:target_index+1]), hyp=10**last_lam)
last_bc = np.argmin(last_ubic)
while now_lam > 0:
    now_ubic = UBIC(b_bics, b_uns, len(y_pre[:, target_index:target_index+1]), hyp=10**now_lam)
    now_bc = np.argmin(now_ubic)
    
    diff_com = now_bc-last_bc
    imp = abs((b_bics[now_bc]-b_bics[last_bc])/(b_bics[last_bc]*diff_com))
    print(min(last_bc, now_bc), '<--->', max(last_bc, now_bc), np.nan_to_num(imp, nan=np.inf))
    
    if (diff_com > 0 and imp < thres) or (diff_com < 0 and imp > thres):
        break
    
    last_lam = now_lam
    now_lam = last_lam-delta
    last_ubic = now_ubic
    last_bc = now_bc

last_lam = round(last_lam, 10)
last_lam, last_ubic, last_bc

4 <---> 4 inf
4 <---> 4 inf


(1.3319703656,
 array([-173871.31467316, -157363.78220475, -201031.85677548,
        -248578.55509579, -325670.03750754, -325570.25215782,
        -325537.96266964, -325520.63274289, -325439.03396138,
        -325284.88168674, -325271.1044624 , -325246.89446851]),
 4)

In [45]:
np.nonzero(miqp3(normalize(X_pre, axis=0), y_pre[:, 1:2].flatten(), 5))

(array([ 2,  5, 10, 16, 24]),)

In [46]:
np.nonzero(miqp2(normalize(X_pre, axis=0), y_pre[:, 1:2].flatten(), 5))

(array([ 2,  4,  5, 16, 24]),)